<a href="https://colab.research.google.com/github/merlynjocol/DigitalActions_NLP_NLU/blob/main/HEIDI_Semantic_Search_Final_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Search

Semantic search seeks to improve search accuracy by understanding the content of the search query. In contrast to traditional search engines which only find documents based on lexical matches, semantic search can also find synonyms.

https://www.sbert.net/examples/applications/semantic-search/README.html

# Background

The idea behind semantic search is to embed all entries in your corpus, whether they be sentences, paragraphs, or documents, into a vector space.

At search time, the query is embedded into the same vector space and the closest embeddings from your corpus are found. These entries should have a high semantic overlap with the query.

SemanticSearch

## Symmetric vs. Asymmetric Semantic Search

A critical distinction for your setup is symmetric vs. asymmetric semantic search:

For symmetric semantic search your query and the entries in your corpus are of about the same length and have the same amount of content. An example would be searching for similar questions: Your query could for example be “How to learn Python online?” and you want to find an entry like “How to learn Python on the web?”. For symmetric tasks, you could potentially flip the query and the entries in your corpus.

For asymmetric semantic search, you usually have a short query (like a question or some keywords) and you want to find a longer paragraph answering the query. An example would be a query like “What is Python” and you wand to find the paragraph “Python is an interpreted, high-level and general-purpose programming language. Python’s design philosophy …”. For asymmetric tasks, flipping the query and the entries in your corpus usually does not make sense.

It is critical that you choose the right model for your type of task.


# SentenceTransformers

This a Python framework. 


--
This package makes the embedding of a sentence easier, and allows also searching in the corpus for the most similar sentences. It is often used in real world applications, and recommended in general.
---



[Sentences Transformers](https://www.sbert.net/)

In [ ]:
# Find Similar Sentences


# Import libraries 

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.0 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 6.8 MB 43.1 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.5 MB/s 
     |████████████████████████████████| 1.2 MB 34.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=af9eb8d6e20fab19a6be216d936ddf2f3a42c3a1d333c29a495155e0f38f40d3
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
#libraries
import warnings
import pandas as pd
import numpy as np
from tqdm import tqdm



In [ ]:
!pip install pysbd

     |████████████████████████████████| 71 kB 3.6 MB/s 


In [ ]:
# Sentence segmentation
from pysbd.utils import PySBDFactory


In [ ]:
# library nlp tasks
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [ ]:
# Data Loading
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


## Data loading


In [ ]:
data = pd.read_csv(f"/gdrive/MyDrive/Colab Notebooks/HEIDI_SCIENTIFIC_ARTICLES/Datasets/fulltext_discussion.csv")

In [ ]:
#rint("Data Shape: {}".format(data.shape))

In [ ]:
# Percentage of missing column values
percent_missing = data.isnull().sum() * 100 / len(data)
percent_missing

Unnamed: 0        0.0
Unnamed: 0.1      0.0
Authors           0.0
Title             0.0
DOI               0.0
Abstract_clean    0.0
fulltext_noref    0.0
discussion        0.0
dtype: float64

There are ❗ 890 articles overall, each one having 6 columns.

Note: I will focuse the analysis on the discussion .This columns contain the discussion and conclusions

In [ ]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Authors', 'Title', 'DOI',
       'Abstract_clean', 'fulltext_noref', 'discussion'],
      dtype='object')

#Look the data 

In [ ]:
data.head(3)

,Unnamed: 0,Unnamed: 0.1,Authors,Title,DOI,Abstract_clean,fulltext_noref,discussion
0,0,0,"Kelly R,Fleming A,Pecl GT",Citizen science and social licence: Improving ...,10.1016/j.ocecoaman.2019.104855,Marine stakeholder groups have diverse relatio...,Citizen science and social licence_ Improving ...,The concept of social licence has many compon...
1,1,1,"Kim G,Newman G,Jiang B",Urban regeneration: Community engagement proce...,10.1016/j.cities.2020.102730,Vacant land presents many challenges for older...,Urban regeneration_ Community engagement proce...,4.1. Understanding the problems and potential...
2,2,2,"Stenhouse A,Perry T,Grützner F,Lewis M,Koh LP",EchidnaCSI – Improving monitoring of a cryptic...,10.1016/j.gecco.2021.e01626,Short-beaked echidna (Tachyglossus aculeatus) ...,EchidnaCSI – Improving monitoring of a cryptic...,4.1. Mobile app use The echidnaCSI app has pr...


# Data Pre-Process


In [ ]:
# -----------------------------
# This in an example in how that looks 
# ------------------------------

In [ ]:
# Remove columns not necesary

dict = {'Abstract_clean': 'abstract',
        'Title': 'title',
        'fulltext_noref': 'body_text'}
 
# call rename () method
data.rename(columns=dict,
          inplace=True)


In [ ]:
df = data.drop(['Unnamed: 0', 'Unnamed: 0.1', "Authors"	], axis=1)

In [ ]:
df.columns

Index(['title', 'DOI', 'abstract', 'body_text', 'discussion'], dtype='object')

In [ ]:
df.head(2)

,title,DOI,abstract,body_text,discussion
0,Citizen science and social licence: Improving ...,10.1016/j.ocecoaman.2019.104855,Marine stakeholder groups have diverse relatio...,Citizen science and social licence_ Improving ...,The concept of social licence has many compon...
1,Urban regeneration: Community engagement proce...,10.1016/j.cities.2020.102730,Vacant land presents many challenges for older...,Urban regeneration_ Community engagement proce...,4.1. Understanding the problems and potential...


In [ ]:
# checking the text (sentences and estructure)
txt = df.iloc[1].discussion
txt


" 4.1. Understanding the problems and potential value of vacant urban land Before people can fully understand the potential ecological and social value of vacant land, cities and residents must understand the current conditions of their vacant properties. Most urban residents do not even know how many vacant properties are in their municipality (although municipal vacant land inventories are increasing globally), which are eligible for tax sale or tax foreclosure, and which are in the process of mortgage foreclosure. In addition, they might be unaware of which vacant properties have contamination issues (e.g. brownfields) that could potentially adversely affect people's health. An inventoried assessment of vacant land conditions can provide detailed descriptions of neighborhood issues using both statistical and qualitative information. Vacant land data should not rely only on statistics or geographic data; they should also include resident surveys to understand their attitudes and perc

## Sentence Segmentation

The library spaCy bring the capability to segments Doc objects into sentences. This task is known as sentence segmentation.

Sentence segmentation imposes additional structure to larger texts. By determining the boundaries of a sentence, we can constrain tasks such as dependency parsing to individual sentences.

spaCy provides access to the results of sentence segmentation via the attribute sents of a Doc object.

Let's loop over the sentences contained in the Doc object doc and count them using Python's enumerate() function.

Using the enumerate() function returns a count that increases with each item in the loop.

We assign this count to the variable number, whereas each sentence is stored under sent. We then print out both at the same time using the print() function.

### Using the pySBD - python Sentence Boundary Disambiguation [Sentence from SpaCy](https://https://spacy.io/universe/project/python-sentence-boundary-disambiguation)

pySBD is 'real-world' sentence segmenter which extracts reasonable sentences when the format and domain of the input text are unknown. It is a rules-based algorithm based on The Golden Rules - a set of tests to check accuracy of segmenter in regards to edge case scenarios developed by TM-Town dev team. pySBD is python port of ruby gem Pragmatic Segmenter.

- Paper: https://arxiv.org/abs/2010.09657

- Golden Rules https://s3.amazonaws.com/tm-town-nlp-resources/golden_rules.txt


### Creating the container (NLP)

First we created a doc from the text, which is a container for a document and all of its annotations. Then we iterated through the document to see what spaCy had parsed.

---



*Let*'s run each document through the natural language parser (nlp variable)!:

In [ ]:
# Rule-based sentence boundary detection 
nlp = spacy.blank('en')
nlp.add_pipe(PySBDFactory(nlp))

In [ ]:
%%time
df['sentences_SpaCy'] = df["discussion"].apply(lambda x: [sent.text for sent in nlp(x).sents])

CPU times: user 10.1 s, sys: 27.6 ms, total: 10.2 s
Wall time: 10.2 s


In [ ]:
df.head(2)

,title,DOI,abstract,body_text,discussion,sentences_SpaCy
0,Citizen science and social licence: Improving ...,10.1016/j.ocecoaman.2019.104855,Marine stakeholder groups have diverse relatio...,Citizen science and social licence_ Improving ...,The concept of social licence has many compon...,"[ , The concept of social licence has many com..."
1,Urban regeneration: Community engagement proce...,10.1016/j.cities.2020.102730,Vacant land presents many challenges for older...,Urban regeneration_ Community engagement proce...,4.1. Understanding the problems and potential...,"[ , 4.1., Understanding the problems and poten..."


In [ ]:
# sentence segmentation verification 
b= df.iloc[1].sentences_SpaCy
b

[' ',
 '4.1.',
 'Understanding the problems and potential value of vacant urban land Before people can fully understand the potential ecological and social value of vacant land, cities and residents must understand the current conditions of their vacant properties.',
 'Most urban residents do not even know how many vacant properties are in their municipality (although municipal vacant land inventories are increasing globally), which are eligible for tax sale or tax foreclosure, and which are in the process of mortgage foreclosure.',
 "In addition, they might be unaware of which vacant properties have contamination issues (e.g. brownfields) that could potentially adversely affect people's health.",
 'An inventoried assessment of vacant land conditions can provide detailed descriptions of neighborhood issues using both statistical and qualitative information.',
 'Vacant land data should not rely only on statistics or geographic data; they should also include resident surveys to understan

# Preparing the datasets

In [ ]:
# removing columns 
dfsentences = df.drop(['body_text', 'discussion'], axis=1)

In [ ]:
dfsentences.columns

Index(['title', 'DOI', 'abstract', 'sentences_SpaCy'], dtype='object')

In [ ]:
dict = {'sentences_SpaCy': 'sentences'}
 
# call rename () method
dfsentences.rename(columns=dict, inplace=True)

In [ ]:
text_dict = dfsentences.to_dict()
len_text = len(text_dict["DOI"])

In [ ]:
DOI_list  = []
title_list = []
abstract_list = []
sentences_list = []

for i in tqdm(range(0,len_text)):
  DOI = text_dict["DOI"][i]
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  sentences = text_dict["sentences"][i]
  
  for s in sentences:
    DOI_list.append(DOI)
    title_list.append(title)
    abstract_list.append(abstract)
    sentences_list.append(s)



100%|██████████| 20/20 [00:00<00:00, 7239.05it/s]


In [ ]:
df_sentences = pd.DataFrame({"DOI":DOI_list},index=sentences_list)
df_sentences.to_csv("/gdrive/MyDrive/Colab Notebooks/HEIDI_SCIENTIFIC_ARTICLES/Datasets/sentences_seg.csv")


In [ ]:
df_sentences.shape

(2211, 1)

In [ ]:
df_sentences_segfull = pd.DataFrame({"DOI":DOI_list,"title":title_list,"abstract":abstract_list},index=sentences_list)
df_sentences_segfull.to_csv("/gdrive/MyDrive/Colab Notebooks/HEIDI_SCIENTIFIC_ARTICLES/Datasets/sentences_segfull.csv")


In [ ]:
df_sentences_segfull 

,DOI,title,abstract
,10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"The concept of social licence has many components (Parsons and Moffat, 2014) and in this study, we focused on trust, legitimacy and relationships.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"We identified the potential for citizen science to be involved in creating and building social licence, but not necessarily the outcomes of achieving it or maintaining it.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
The results of this case-study demonstrate that marine citizen science i.,10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"provides opportunity for marine users to connect with one another, ii.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
...,...,...,...
"Funding This research was supported by the NIH through the HEAL Initiative, under award number: UM1DA049415 (New York) and UM1DA049394 (RTI).",10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...
Contributors NE and BR led the overall conceptualization of the paper.,10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...
All authors contributed to the theory and conceptualization of the paper and the writing of the manuscript.,10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...
All authors provided critical feedback to shape the data-driven paradigm & systems science approach and give final approval of the version to be published.,10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...


Preparing Data for Embedding


In [ ]:
import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/HEIDI_SCIENTIFIC_ARTICLES/Datasets/sentences_seg.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")

In [ ]:
df_sentences.head(10)

,DOI
Unnamed: 0,
,10.1016/j.ocecoaman.2019.104855
"The concept of social licence has many components (Parsons and Moffat, 2014) and in this study, we focused on trust, legitimacy and relationships.",10.1016/j.ocecoaman.2019.104855
"We identified the potential for citizen science to be involved in creating and building social licence, but not necessarily the outcomes of achieving it or maintaining it.",10.1016/j.ocecoaman.2019.104855
The results of this case-study demonstrate that marine citizen science i.,10.1016/j.ocecoaman.2019.104855
"provides opportunity for marine users to connect with one another, ii.",10.1016/j.ocecoaman.2019.104855
"can improve their understanding of other user groups, and iii.",10.1016/j.ocecoaman.2019.104855
"can reveal their shared values of the marine environment, towards improving different user groups' perceptions and acceptance of one another's activities.",10.1016/j.ocecoaman.2019.104855
We have shown that participation in programmes such as Redmap can improve perceptions of trustworthiness amongst marine user groups.,10.1016/j.ocecoaman.2019.104855
"Social licence has popular appeal because it conveys the notion of community acceptance and support (Bursey and Whiting, 2015).",10.1016/j.ocecoaman.2019.104855


In [ ]:
df_sentences = df_sentences["DOI"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

2080

In [ ]:
list(df_sentences.keys())[:2]

[' ',
 'The concept of social licence has many components (Parsons and Moffat, 2014) and in this study, we focused on trust, legitimacy and relationships.']

In [ ]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 2080/2080 [00:00<00:00, 815875.09it/s]


# Bring full dataframe

In [ ]:
df_allinfo = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/HEIDI_SCIENTIFIC_ARTICLES/Datasets/sentences_segfull.csv", index_col=0)
df_allinfo.head(2)

,DOI,title,abstract
,10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"The concept of social licence has many components (Parsons and Moffat, 2014) and in this study, we focused on trust, legitimacy and relationships.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...


## Transformers

This is a simple application for sentence embeddings: semantic search. 
 
I have a corpus with various sentences. Then, for a given query sentence, I want to find the most similar sentence in this corpus.

This script outputs for various queries the top 5 most similar sentences in the corpus.

# Models 

This framework provides an easy method to compute dense vector representations for sentences, paragraphs, and images. The models are based on transformer networks like BERT / RoBERTa / XLM-RoBERTa etc. and achieve state-of-the-art performance in various task. Text is embedding in vector space such that similar text is close and can efficiently be found using cosine similarity.

UKL provide an increasing number of state-of-the-art pretrained models for more than 100 languages, fine-tuned for various use-cases.

https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

For the full documentation, see www.SBERT.net.

The following publications are integrated in this framework:

Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks (https://arxiv.org/abs/1908.10084)


In [ ]:
# call transformers
from sentence_transformers import SentenceTransformer
import scipy.spatial
#import pickle as pkl



# Models to use

This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.


In [ ]:
#original model
model1 = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

## multi-qa-MiniLM-L6-cos-v1

This is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and was designed for semantic search. It has been trained on 215M (question, answer) pairs from diverse sources. For an introduction to semantic search, have a look at: SBERT.net - Semantic Search

In [ ]:
# model 2
model2 = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

https://huggingface.co/allenai/scibert_scivocab_uncased

https://github.com/allenai/scibert

## SciBERT is a BERT model trained on scientific text.

SciBERT is trained on papers from the corpus of semanticscholar.org. Corpus size is 1.14M papers, 3.1B tokens. We use the full text of the papers in training, not just abstracts.

SciBERT has its own vocabulary (scivocab) that's built to best match the training corpus. We trained cased and uncased versions. We also include models trained on the original BERT vocabulary (basevocab) for comparison.

It results in state-of-the-art performance on a wide range of scientific domain nlp tasks. The details of the evaluation are in the paper. Evaluation code and data are included in this repo.

In [ ]:
# model 3
model3 = SentenceTransformer('allenai/scibert_scivocab_uncased')

Downloading:   0%|          | 0.00/437 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/allenai_scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Model Embeddings

## Corpus

## Corpus embeddings

In [ ]:
# only Corpus with  sentences
corpus = df_sentences_list

In [ ]:
# embeddings model 1 ('bert-base-nli-mean-tokens')
corpus_embeddings1 = model1.encode(corpus,show_progress_bar=True)

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
# embeddings model 2  ('multi-qa-MiniLM-L6-cos-v1')
corpus_embeddings2 = model2.encode(corpus,show_progress_bar=True)

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
import numpy as np
import torch

In [ ]:
# embeddings model 3 SciBert ('allenai/scibert_scivocab_uncased')
corpus_embeddings3 = model3.encode(corpus,convert_to_tensor=True , show_progress_bar=True)

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

## Query sentences

In [ ]:
# Query sentences to try:
queries = ['concept of social licence is trust', 
           "incentives for community engagement", 
           "increase the participation of communities and people"]

# Queries embeddings Model 1

In [ ]:
query_embeddings1 = model1.encode(queries,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Calculating the similarity Model 1




In [ ]:
from pprint import pprint

In [ ]:
#Find the closest 1 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings1):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings1, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df_allinfo.loc[df_allinfo.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["DOI"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences in corpus:
==========================Query==============================
=== concept of social licence is trust =====
Score:    (Score: 0.7443) 

Paragraph:    These events cultivate social capital, trust, and networks (Roetman and Daniels, 2011; Putnam et al., 1993). 

paper_id:   10.1016/j.ufug.2021.127054 

Title:   The relationship between knowledge and community engagement in local urban forest governance: A case study examining the role of resident association members in Mississauga, Canada 

Abstract:   Urban forests, integral to a city’s critical infrastructure, are traditionally under the mandate of local governments, yet in reality, the decision-making for their conservation is influenced by a myriad of factors operating at the neighbourhood level. In some neighbourhoods, decisions are heavily influenced by formal Resident Associations (RAs). Using a case study approach, in-depth interviews were conducted with selected engaged and committed RA me

# Queries embeddings Model 2

In [ ]:
query_embeddings2 = model2.encode(queries,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Calculating the similarity Model 2




In [ ]:
#Find the closest 1 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings2):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings2, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df_allinfo.loc[df_allinfo.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["DOI"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences in corpus:
==========================Query==============================
=== concept of social licence is trust =====
Score:    (Score: 0.8747) 

Paragraph:    The concept of social licence has many components (Parsons and Moffat, 2014) and in this study, we focused on trust, legitimacy and relationships. 

paper_id:   10.1016/j.ocecoaman.2019.104855 

Title:   Citizen science and social licence: Improving perceptions and connecting marine user groups 

Abstract:   Marine stakeholder groups have diverse relationships with the ocean and life within it, which can create conflict and distrust between them. Citizen science and social licence present promising means to develop dialogue between these diverse marine stakeholders and improve outcomes for marine management. Citizen science can be defined as public engagement in scientific research and activities and amongst other benefits, has been demonstrated to improve communication and relationships amongst res

# Queries embeddings Model 3

In [ ]:
query_embeddings3 = model3.encode(queries,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Calculating the similarity Model 3 SciBERT

---






In [ ]:
#Find the closest 1 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings3):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings3, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df_allinfo.loc[df_allinfo.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["DOI"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences in corpus:
==========================Query==============================
=== concept of social licence is trust =====
Score:    (Score: 0.8207) 

Paragraph:    Social licence is a dynamic concept and in this instance, is a continuous process of repeated interaction and positive engagement with marine users and stakeholders. 

paper_id:   10.1016/j.ocecoaman.2019.104855 

Title:   Citizen science and social licence: Improving perceptions and connecting marine user groups 

Abstract:   Marine stakeholder groups have diverse relationships with the ocean and life within it, which can create conflict and distrust between them. Citizen science and social licence present promising means to develop dialogue between these diverse marine stakeholders and improve outcomes for marine management. Citizen science can be defined as public engagement in scientific research and activities and amongst other benefits, has been demonstrated to improve communication and relati

# Getting the embeddings 


In [ ]:
df_sentences_segfull

,DOI,title,abstract
,10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"The concept of social licence has many components (Parsons and Moffat, 2014) and in this study, we focused on trust, legitimacy and relationships.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"We identified the potential for citizen science to be involved in creating and building social licence, but not necessarily the outcomes of achieving it or maintaining it.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
The results of this case-study demonstrate that marine citizen science i.,10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
"provides opportunity for marine users to connect with one another, ii.",10.1016/j.ocecoaman.2019.104855,Citizen science and social licence: Improving ...,Marine stakeholder groups have diverse relatio...
...,...,...,...
"Funding This research was supported by the NIH through the HEAL Initiative, under award number: UM1DA049415 (New York) and UM1DA049394 (RTI).",10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...
Contributors NE and BR led the overall conceptualization of the paper.,10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...
All authors contributed to the theory and conceptualization of the paper and the writing of the manuscript.,10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...
All authors provided critical feedback to shape the data-driven paradigm & systems science approach and give final approval of the version to be published.,10.1016/j.drugalcdep.2021.108675,Using community engagement to implement eviden...,Community-driven responses are essential to en...


In [ ]:
df_visual = pd.read_csv("/gdrive/MyDrive/Colab Notebooks/HEIDI_SCIENTIFIC_ARTICLES/Datasets/sentences_seg.csv")


In [ ]:
df_visual.columns

Index(['Unnamed: 0', 'DOI'], dtype='object')

In [ ]:
df_visual

,Unnamed: 0,DOI
0,,10.1016/j.ocecoaman.2019.104855
1,The concept of social licence has many compone...,10.1016/j.ocecoaman.2019.104855
2,We identified the potential for citizen scienc...,10.1016/j.ocecoaman.2019.104855
3,The results of this case-study demonstrate tha...,10.1016/j.ocecoaman.2019.104855
4,provides opportunity for marine users to conne...,10.1016/j.ocecoaman.2019.104855
...,...,...
2206,Funding This research was supported by the NIH...,10.1016/j.drugalcdep.2021.108675
2207,Contributors NE and BR led the overall concept...,10.1016/j.drugalcdep.2021.108675
2208,All authors contributed to the theory and conc...,10.1016/j.drugalcdep.2021.108675
2209,All authors provided critical feedback to shap...,10.1016/j.drugalcdep.2021.108675


In [ ]:

sentence_bert_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def get_embeddings(sentences):
    return sentence_bert_model.encode(sentences,
                                    batch_size=32, 
                                    show_progress_bar=True)

In [ ]:
e = get_embeddings(df_visual['Unnamed: 0'])

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

## Exporting to Embedding Projector FormatPermalink
Embedding Projector requires two TSV files to load our custom embeddings.

output.tsv: This file should contain the embeddings without any headers.
metadata.tsv: This file should contain the original text and labels for the embeddings
Let’s first generate the output.tsv file for our sentence embeddings from the previous step.

In [ ]:
# Convert NumPy array of embedding into data frame
embedding_df = pd.DataFrame(e)

# Save dataframe as as TSV file without any index and header
embedding_df.to_csv('output3.tsv', sep='\t', index=None, header=None)

In [ ]:
# Save dataframe without any index
df_visual.to_csv('metadata2.tsv', index=False, sep='\t')

# Testing


In [ ]:
# Query sentences to try:
query = ["generational digital divide
"]

SyntaxError: ignored

In [ ]:
query_embeddings4 = model3.encode(query,show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#Find the closest 1 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(query, query_embeddings4):
    distances = scipy.spatial.distance.cdist([query_embeddings4], corpus_embeddings3, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df_allinfo.loc[df_allinfo.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["DOI"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences in corpus:
==========================Query==============================
=== t =====
Score:    (Score: 0.7555) 

Paragraph:    Assessing the financial requirements and sources to implement plans is essential to the community engagement process. 

paper_id:   10.1016/j.cities.2020.102730 

Title:   Urban regeneration: Community engagement process for vacant land in declining cities 

Abstract:   Vacant land presents many challenges for older financially distressed cities. Community engagement is a very important element to solve the urban vacant land problem and assist in long term regeneration. This paper reviews what plans, policies, implementation methods, and community engagement process were developed to overcome barriers and challenges to vacant land projects. Most studies reveal that the importance of community engagement process in terms of understanding the problems and potential value of vacant land, redevelopment process, financial support, regul